### Merge channels and Timesteps
Format the scheduled spikes appropriately, such that the VHDL testbench can read data

In [ ]:
import os
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def read_file(filename):
    with open(filename, 'r') as file:
        data = file.readlines()
    return data


# Parse the data into a dictionary of channels
def parse_data(data):
    channels = {}
    current_channel = None

    for line in data:
        if line.startswith("Channel"):
            current_channel = int(line.split()[1].strip(':'))
            channels[current_channel] = []
        elif current_channel is not None:
            spikes = list(map(int, line.split()))
            channels[current_channel].append(spikes)

    return channels


# Transform the data into the desired format
def transform_data(channels):
    transformed = []
    num_channels = len(channels)
    num_cycles = len(channels[0])

    for i in range(num_cycles):
        for j in range(len(channels[0][i])):
            row = []
            for ch in range(num_channels):
                row.append(channels[ch][i][j])
            transformed.append(row)

    return transformed

def write_file(filename, data):
    with open(filename, 'w') as file:
        for row in data:
            file.write(''.join(map(str, row)) + '\n')

# Function to write the data to an output file
def write_combined_timestep_data(combined_data, output_filename):
    with open(output_filename, 'w') as file:
        num_cycles = len(combined_data[0])
        num_time_steps = len(combined_data)

        for cycle in range(num_cycles // 9):
            for timestep in range(num_time_steps):
                start_line = cycle * 9
                end_line = start_line + 9
                for row in combined_data[timestep][start_line:end_line]:
                    file.write(''.join(map(str, row)) + '\n')
                # file.write('\n')


In [ ]:
base_path = '/content/drive/My Drive/project_aug/v4'
timesteps = 3

combined_data = []
combined_filename = base_path + '/input_output_v5/insp_8b_tsmerged_vhdl.txt'

for timestep in range(timesteps):
  input_filename = base_path + f'/input_output_v5/inspk_8b_schedld{timestep}.txt'
  output_filename = base_path + f'/input_output_v5/inspk_8b_vhdl{timestep}.txt'

  data = read_file(input_filename)
  channels = parse_data(data)
  transformed_data = transform_data(channels)
  combined_data.append(transformed_data)
  # Write the transformed data to indvidual files
  write_file(output_filename, transformed_data)

write_combined_timestep_data(combined_data, combined_filename)